# Day 3-3: 종합 실습 - 로그 분석 자동화 봇 (정답)

## 학습 목표
- 3일간 배운 모든 개념을 통합하여 실전 프로젝트 완성
- glob으로 파일 탐색, re로 패턴 추출, 제너레이터로 효율적 처리
- 클래스 기반 설계와 예외 처리를 적용한 견고한 코드 작성
- JSON으로 결과 저장 및 로깅으로 프로세스 추적

## 프로젝트 개요
서버 로그 파일들을 자동으로 분석하여 에러와 경고를 추출하고,
통계를 생성한 뒤 JSON 리포트로 저장하는 자동화 봇을 만듭니다.

### 통합 개념
- **Day 1**: Collections (Counter, defaultdict), comprehensions
- **Day 2**: Generators, file I/O, JSON, modules
- **Day 3**: OOP classes, exception handling, logging, regex, glob, datetime

---
## Task 1: ⭐ Setup - 샘플 로그 파일 생성

먼저 분석할 샘플 로그 파일들을 생성합니다.

In [ ]:
import os
from datetime import datetime, timedelta

# 로그 파일을 저장할 디렉토리 생성
log_dir = "server_logs"
os.makedirs(log_dir, exist_ok=True)

# 샘플 로그 데이터 (다양한 로그 레벨 포함)
sample_logs = [
    "2025-01-15 10:23:45 INFO Server started successfully",
    "2025-01-15 10:24:12 ERROR Database connection failed: timeout",
    "2025-01-15 10:25:33 WARNING Memory usage at 85%",
    "2025-01-15 10:26:01 INFO User login: user123",
    "2025-01-15 10:27:18 ERROR File not found: /data/config.json",
    "2025-01-15 10:28:45 WARNING Disk space low: 10% remaining",
    "2025-01-15 10:29:22 INFO API request processed in 120ms",
    "2025-01-15 10:30:55 ERROR Authentication failed for user: admin",
]

# 3개의 로그 파일 생성
dates = ["2025_01_15", "2025_01_16", "2025_01_17"]

for date in dates:
    file_path = os.path.join(log_dir, f"app_{date}.log")
    
    with open(file_path, 'w', encoding='utf-8') as f:
        # 각 날짜에 맞게 로그 타임스탬프 수정
        for log_line in sample_logs * 3:  # 로그를 3번 반복하여 충분한 데이터 생성
            # 날짜 부분만 변경
            modified_log = log_line.replace("2025-01-15", date.replace("_", "-"))
            f.write(modified_log + "\n")

print(f"✅ {len(dates)}개의 로그 파일이 생성되었습니다.")
print(f"📁 디렉토리: {log_dir}/")

# 생성된 파일 확인
for date in dates:
    file_path = os.path.join(log_dir, f"app_{date}.log")
    file_size = os.path.getsize(file_path)
    print(f"  - app_{date}.log ({file_size} bytes)")

---
## Task 2: ⭐⭐ File Discovery - glob으로 로그 파일 찾기

glob 모듈을 사용하여 특정 패턴의 로그 파일들을 찾습니다.

In [ ]:
import glob

# server_logs 디렉토리에서 .log 확장자를 가진 모든 파일 찾기
log_files = glob.glob("server_logs/*.log")

# 파일명 순으로 정렬
log_files.sort()

print(f"발견된 로그 파일: {len(log_files)}개")
for file in log_files:
    print(f"  - {file}")

# 추가: 특정 날짜 패턴으로 필터링
jan_15_logs = glob.glob("server_logs/*2025_01_15.log")
print(f"\n2025년 1월 15일 로그: {len(jan_15_logs)}개")

---
## Task 3: ⭐⭐⭐ Pattern Extraction - 정규식으로 로그 파싱

정규식을 사용하여 로그 라인에서 타임스탬프, 레벨, 메시지를 추출합니다.

In [ ]:
import re

# 로그 패턴: YYYY-MM-DD HH:MM:SS LEVEL Message
# 그룹 1: 날짜시간 (\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})
# 그룹 2: 레벨 (INFO|WARNING|ERROR)
# 그룹 3: 메시지 (.+)
log_pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}) (INFO|WARNING|ERROR) (.+)"

# 패턴 컴파일 (성능 향상)
compiled_pattern = re.compile(log_pattern)

# 테스트
test_lines = [
    "2025-01-15 10:24:12 ERROR Database connection failed: timeout",
    "2025-01-15 10:25:33 WARNING Memory usage at 85%",
    "2025-01-15 10:26:01 INFO User login: user123",
    "Invalid log line",  # 매칭 실패 케이스
]

print("로그 파싱 테스트:")
for line in test_lines:
    match = compiled_pattern.match(line)
    
    if match:
        timestamp, level, message = match.groups()
        print(f"\n원본: {line}")
        print(f"  📅 타임스탬프: {timestamp}")
        print(f"  🏷️  레벨: {level}")
        print(f"  💬 메시지: {message}")
    else:
        print(f"\n❌ 패턴 매칭 실패: {line}")

# 추가: 특정 레벨만 필터링하는 패턴
error_pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}) ERROR (.+)"
print("\n\nERROR만 추출:")
for line in test_lines:
    if re.match(error_pattern, line):
        print(f"  🔴 {line}")

---
## Task 4: ⭐⭐⭐ Generator - 메모리 효율적 로그 처리

제너레이터를 사용하여 대용량 로그 파일을 메모리 효율적으로 처리합니다.

In [ ]:
def parse_log_file(file_path, pattern):
    """
    로그 파일을 한 줄씩 읽어 파싱 결과를 yield하는 제너레이터
    
    Args:
        file_path: 로그 파일 경로
        pattern: 정규식 패턴 (컴파일된 객체 또는 문자열)
    
    Yields:
        dict: {'timestamp': str, 'level': str, 'message': str}
    """
    # 패턴이 문자열이면 컴파일
    if isinstance(pattern, str):
        pattern = re.compile(pattern)
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()  # 앞뒤 공백 제거
            
            if not line:  # 빈 줄 건너뛰기
                continue
            
            match = pattern.match(line)
            
            if match:
                timestamp, level, message = match.groups()
                yield {
                    'timestamp': timestamp,
                    'level': level,
                    'message': message,
                    'file': file_path,
                    'line_number': line_num
                }
            else:
                # 매칭 실패한 라인은 건너뛰거나 경고 로그 출력 가능
                pass

# 테스트
if log_files:
    print("제너레이터 테스트:")
    log_gen = parse_log_file(log_files[0], compiled_pattern)
    
    print("\n첫 5개 로그 엔트리:")
    for i, entry in enumerate(log_gen):
        if i >= 5:
            break
        print(f"  {i+1}. [{entry['level']}] {entry['timestamp']} - {entry['message'][:50]}...")
    
    # 제너레이터 체이닝: ERROR만 필터링
    print("\nERROR 로그만 필터링:")
    error_gen = (entry for entry in parse_log_file(log_files[0], compiled_pattern) 
                 if entry['level'] == 'ERROR')
    
    for i, entry in enumerate(error_gen, 1):
        if i > 3:
            break
        print(f"  {i}. {entry['timestamp']} - {entry['message']}")

# 제너레이터의 장점 확인
print("\n💡 제너레이터 장점:")
print("  - 메모리 효율: 전체 파일을 메모리에 로드하지 않음")
print("  - Lazy evaluation: 필요한 만큼만 처리")
print("  - 대용량 파일 처리에 이상적")

---
## Task 5: ⭐⭐⭐⭐ LogAnalyzer Class - OOP 기반 분석기

클래스를 사용하여 로그 분석 로직을 캡슐화합니다.

In [ ]:
from collections import Counter, defaultdict

class LogAnalyzer:
    """
    로그 파일을 분석하여 통계를 생성하는 클래스
    """
    
    def __init__(self, log_pattern):
        """
        Args:
            log_pattern: 로그 파싱을 위한 정규식 패턴
        """
        self.pattern = log_pattern if isinstance(log_pattern, re.Pattern) else re.compile(log_pattern)
        self.level_counter = Counter()  # 로그 레벨별 카운트
        self.errors = []  # ERROR 로그 목록
        self.warnings = []  # WARNING 로그 목록
        self.daily_stats = defaultdict(lambda: defaultdict(int))  # 날짜별 레벨 통계
        self.processed_files = []  # 처리된 파일 목록
        self.total_lines = 0  # 전체 처리된 라인 수
    
    def analyze_file(self, file_path):
        """
        단일 로그 파일 분석
        
        Args:
            file_path: 로그 파일 경로
        
        Returns:
            int: 처리된 로그 엔트리 수
        """
        entry_count = 0
        
        for entry in parse_log_file(file_path, self.pattern):
            # 레벨별 카운트
            self.level_counter[entry['level']] += 1
            
            # 날짜 추출 (YYYY-MM-DD)
            date = entry['timestamp'].split()[0]
            self.daily_stats[date][entry['level']] += 1
            
            # ERROR와 WARNING 별도 저장
            if entry['level'] == 'ERROR':
                self.errors.append({
                    'timestamp': entry['timestamp'],
                    'message': entry['message'],
                    'file': file_path
                })
            elif entry['level'] == 'WARNING':
                self.warnings.append({
                    'timestamp': entry['timestamp'],
                    'message': entry['message'],
                    'file': file_path
                })
            
            entry_count += 1
            self.total_lines += 1
        
        self.processed_files.append(file_path)
        return entry_count
    
    def analyze_directory(self, directory_pattern):
        """
        디렉토리 내 모든 로그 파일 분석
        
        Args:
            directory_pattern: glob 패턴 (예: "logs/*.log")
        
        Returns:
            dict: 파일별 처리 결과
        """
        files = glob.glob(directory_pattern)
        results = {}
        
        for file_path in files:
            count = self.analyze_file(file_path)
            results[file_path] = count
        
        return results
    
    def get_summary(self):
        """
        분석 결과 요약 반환
        
        Returns:
            dict: 통계 요약
        """
        return {
            "total_logs": sum(self.level_counter.values()),
            "total_lines_processed": self.total_lines,
            "files_processed": len(self.processed_files),
            "level_counts": dict(self.level_counter),
            "error_count": len(self.errors),
            "warning_count": len(self.warnings),
            "info_count": self.level_counter.get('INFO', 0),
            "daily_stats": {date: dict(stats) for date, stats in self.daily_stats.items()},
            "top_errors": self.errors[:5],  # 상위 5개 에러
            "top_warnings": self.warnings[:5],  # 상위 5개 경고
            "processed_files": self.processed_files
        }
    
    def get_error_summary(self):
        """
        에러 메시지 패턴 분석
        
        Returns:
            Counter: 에러 유형별 빈도
        """
        # 에러 메시지의 첫 단어로 그룹화
        error_types = Counter()
        for error in self.errors:
            # 메시지의 첫 부분 추출
            first_word = error['message'].split(':')[0] if ':' in error['message'] else error['message'].split()[0]
            error_types[first_word] += 1
        return error_types

# 테스트
print("LogAnalyzer 클래스 테스트:\n")
analyzer = LogAnalyzer(compiled_pattern)

# 디렉토리 전체 분석
results = analyzer.analyze_directory("server_logs/*.log")

print("파일별 처리 결과:")
for file, count in results.items():
    print(f"  {file}: {count}개 로그 엔트리")

# 요약 정보
summary = analyzer.get_summary()
print(f"\n전체 통계:")
print(f"  전체 로그: {summary['total_logs']}개")
print(f"  레벨별: {summary['level_counts']}")
print(f"  ERROR: {summary['error_count']}개")
print(f"  WARNING: {summary['warning_count']}개")

# 에러 패턴 분석
error_types = analyzer.get_error_summary()
print(f"\n에러 유형별 빈도:")
for error_type, count in error_types.most_common(3):
    print(f"  {error_type}: {count}회")

---
## Task 6: ⭐⭐⭐⭐ Statistics & Reporting - 통계 집계 및 리포트

분석 결과를 JSON으로 저장하고 시각적으로 출력합니다.

In [ ]:
import json
from datetime import datetime

def save_report(summary, output_file="log_report.json"):
    """
    분석 결과를 JSON 파일로 저장
    
    Args:
        summary: 분석 요약 딕셔너리
        output_file: 출력 파일명
    
    Returns:
        str: 저장된 파일 경로
    """
    report = {
        "metadata": {
            "generated_at": datetime.now().isoformat(),
            "version": "1.0",
            "analyzer": "LogAnalyzerBot"
        },
        "analysis": summary
    }
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(report, f, indent=2, ensure_ascii=False)
    
    return output_file

def print_report(summary):
    """
    분석 결과를 콘솔에 출력
    
    Args:
        summary: 분석 요약 딕셔너리
    """
    print("\n" + "="*60)
    print("📊 로그 분석 리포트")
    print("="*60)
    
    # 전체 요약
    print(f"\n📈 전체 통계")
    print(f"  총 로그 엔트리: {summary['total_logs']:,}개")
    print(f"  처리된 파일: {summary['files_processed']}개")
    
    # 레벨별 카운트
    print(f"\n🏷️  로그 레벨별 분포")
    level_counts = summary['level_counts']
    total = sum(level_counts.values())
    
    for level, count in sorted(level_counts.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / total * 100) if total > 0 else 0
        bar = "█" * int(percentage / 2)  # 간단한 막대 그래프
        print(f"  {level:8s}: {count:4d} ({percentage:5.1f}%) {bar}")
    
    # 에러와 경고
    print(f"\n⚠️  중요 이슈")
    print(f"  ERROR: {summary['error_count']}개")
    print(f"  WARNING: {summary['warning_count']}개")
    
    # 상위 에러
    if summary['top_errors']:
        print(f"\n🔴 상위 ERROR 로그 (최대 5개)")
        for i, error in enumerate(summary['top_errors'], 1):
            print(f"  {i}. [{error['timestamp']}] {error['message'][:60]}...")
    
    # 상위 경고
    if summary['top_warnings']:
        print(f"\n🟡 상위 WARNING 로그 (최대 5개)")
        for i, warning in enumerate(summary['top_warnings'], 1):
            print(f"  {i}. [{warning['timestamp']}] {warning['message'][:60]}...")
    
    # 날짜별 통계
    if summary['daily_stats']:
        print(f"\n📅 날짜별 통계")
        for date, stats in sorted(summary['daily_stats'].items()):
            print(f"  {date}:")
            for level, count in sorted(stats.items()):
                print(f"    {level}: {count}개")
    
    print("\n" + "="*60)

def generate_html_report(summary, output_file="log_report.html"):
    """
    HTML 형식의 리포트 생성 (보너스)
    
    Args:
        summary: 분석 요약 딕셔너리
        output_file: 출력 HTML 파일명
    """
    html_template = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>로그 분석 리포트</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            h1 {{ color: #333; }}
            .stat {{ background: #f0f0f0; padding: 10px; margin: 10px 0; border-radius: 5px; }}
            .error {{ color: #d32f2f; }}
            .warning {{ color: #f57c00; }}
            .info {{ color: #1976d2; }}
        </style>
    </head>
    <body>
        <h1>📊 로그 분석 리포트</h1>
        <div class="stat">
            <h2>전체 통계</h2>
            <p>총 로그: {summary['total_logs']:,}개</p>
            <p>처리된 파일: {summary['files_processed']}개</p>
        </div>
        <div class="stat">
            <h2>레벨별 분포</h2>
            <p class="error">ERROR: {summary['error_count']}개</p>
            <p class="warning">WARNING: {summary['warning_count']}개</p>
            <p class="info">INFO: {summary['info_count']}개</p>
        </div>
    </body>
    </html>
    """
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(html_template)
    
    return output_file

# 테스트
summary = analyzer.get_summary()
print_report(summary)

# JSON 저장
json_file = save_report(summary)
print(f"\n💾 JSON 리포트 저장: {json_file}")

# HTML 리포트 생성 (보너스)
html_file = generate_html_report(summary)
print(f"💾 HTML 리포트 저장: {html_file}")

---
## Task 7: ⭐⭐⭐⭐⭐ Complete Automation - 완전 자동화 파이프라인

예외 처리, 로깅, 그리고 모든 기능을 통합한 완전한 자동화 봇을 구현합니다.

In [ ]:
import logging
from pathlib import Path

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('log_analyzer.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class LogAnalyzerBot:
    """
    완전 자동화된 로그 분석 봇
    예외 처리, 로깅, 리포팅을 모두 포함
    """
    
    def __init__(self, log_directory, output_dir="reports"):
        """
        Args:
            log_directory: 로그 파일이 있는 디렉토리
            output_dir: 리포트를 저장할 디렉토리
        """
        self.log_directory = Path(log_directory)
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
        
        # 로그 패턴
        self.log_pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}) (INFO|WARNING|ERROR) (.+)"
        self.analyzer = None
    
    def validate_log_directory(self):
        """
        로그 디렉토리 유효성 검사
        
        Returns:
            bool: 유효성 여부
        
        Raises:
            FileNotFoundError: 디렉토리가 존재하지 않을 때
            PermissionError: 읽기 권한이 없을 때
        """
        if not self.log_directory.exists():
            raise FileNotFoundError(f"로그 디렉토리를 찾을 수 없습니다: {self.log_directory}")
        
        if not self.log_directory.is_dir():
            raise NotADirectoryError(f"경로가 디렉토리가 아닙니다: {self.log_directory}")
        
        # 로그 파일 존재 확인
        log_files = list(self.log_directory.glob("*.log"))
        if not log_files:
            logger.warning(f"로그 파일이 없습니다: {self.log_directory}")
            return False
        
        logger.info(f"{len(log_files)}개의 로그 파일 발견")
        return True
    
    def analyze_logs(self):
        """
        로그 분석 실행
        
        Returns:
            dict: 분석 결과 요약
        """
        logger.info("로그 분석 시작")
        
        self.analyzer = LogAnalyzer(self.log_pattern)
        pattern = str(self.log_directory / "*.log")
        
        try:
            results = self.analyzer.analyze_directory(pattern)
            logger.info(f"{len(results)}개 파일 처리 완료")
            
            summary = self.analyzer.get_summary()
            logger.info(f"총 {summary['total_logs']}개 로그 분석 완료")
            
            return summary
            
        except Exception as e:
            logger.error(f"로그 분석 중 오류 발생: {e}", exc_info=True)
            raise
    
    def generate_reports(self, summary):
        """
        리포트 생성 및 저장
        
        Args:
            summary: 분석 결과 요약
        
        Returns:
            dict: 생성된 리포트 파일 경로들
        """
        logger.info("리포트 생성 시작")
        
        # 타임스탬프 포함 파일명
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # JSON 리포트
        json_path = self.output_dir / f"log_report_{timestamp}.json"
        save_report(summary, str(json_path))
        logger.info(f"JSON 리포트 저장: {json_path}")
        
        # HTML 리포트
        html_path = self.output_dir / f"log_report_{timestamp}.html"
        generate_html_report(summary, str(html_path))
        logger.info(f"HTML 리포트 저장: {html_path}")
        
        return {
            'json': str(json_path),
            'html': str(html_path)
        }
    
    def send_alerts(self, summary, error_threshold=10):
        """
        임계값 초과 시 알림 (시뮬레이션)
        
        Args:
            summary: 분석 결과 요약
            error_threshold: 에러 임계값
        """
        error_count = summary['error_count']
        
        if error_count > error_threshold:
            alert_message = f"⚠️  경고: {error_count}개의 에러 감지 (임계값: {error_threshold})"
            logger.warning(alert_message)
            print(f"\n{alert_message}")
            print("실제 환경에서는 이메일/SMS/Slack 알림을 전송할 수 있습니다.")
        else:
            logger.info(f"에러 수 정상 범위: {error_count}/{error_threshold}")
    
    def run(self):
        """
        전체 분석 프로세스 실행
        
        Returns:
            bool: 성공 여부
        """
        try:
            logger.info("="*60)
            logger.info("로그 분석 봇 시작")
            logger.info("="*60)
            
            # 1. 로그 디렉토리 검증
            if not self.validate_log_directory():
                logger.warning("분석할 로그 파일이 없습니다")
                return False
            
            # 2. 로그 분석
            summary = self.analyze_logs()
            
            # 3. 콘솔에 리포트 출력
            print_report(summary)
            
            # 4. 리포트 파일 생성
            report_files = self.generate_reports(summary)
            
            # 5. 알림 체크
            self.send_alerts(summary)
            
            logger.info("="*60)
            logger.info("로그 분석 완료")
            logger.info(f"생성된 리포트: {report_files}")
            logger.info("="*60)
            
            return True
            
        except FileNotFoundError as e:
            logger.error(f"파일을 찾을 수 없습니다: {e}")
            return False
        except PermissionError as e:
            logger.error(f"파일 접근 권한이 없습니다: {e}")
            return False
        except Exception as e:
            logger.error(f"예상치 못한 오류 발생: {e}", exc_info=True)
            return False

# 실행
if __name__ == "__main__":
    print("\n" + "="*60)
    print("🤖 로그 분석 자동화 봇 시작")
    print("="*60 + "\n")
    
    bot = LogAnalyzerBot("server_logs")
    success = bot.run()
    
    if success:
        print("\n" + "="*60)
        print("✅ 로그 분석이 성공적으로 완료되었습니다!")
        print("="*60)
    else:
        print("\n" + "="*60)
        print("❌ 로그 분석 중 오류가 발생했습니다.")
        print("📝 log_analyzer.log 파일을 확인하세요.")
        print("="*60)

---
## 확장 아이디어 구현 예제

### 1. 이메일 알림

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_alert_email(summary, threshold=10):
    """
    에러가 임계값을 초과하면 이메일 발송
    
    Args:
        summary: 분석 결과 요약
        threshold: 에러 임계값
    """
    if summary['error_count'] <= threshold:
        return
    
    # 이메일 설정 (실제 사용 시 환경변수로 관리)
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    sender_email = "your_email@gmail.com"
    sender_password = "your_password"  # 앱 비밀번호 사용 권장
    receiver_email = "admin@example.com"
    
    # 이메일 본문 작성
    message = MIMEMultipart("alternative")
    message["Subject"] = f"⚠️ 로그 알림: {summary['error_count']}개 에러 감지"
    message["From"] = sender_email
    message["To"] = receiver_email
    
    text = f"""
    로그 분석 알림
    
    에러 수: {summary['error_count']}개 (임계값: {threshold})
    경고 수: {summary['warning_count']}개
    전체 로그: {summary['total_logs']}개
    
    상위 에러:
    {chr(10).join(f"- {e['message']}" for e in summary['top_errors'][:3])}
    """
    
    part = MIMEText(text, "plain")
    message.attach(part)
    
    # 이메일 발송 (실제 환경에서만 실행)
    try:
        # with smtplib.SMTP(smtp_server, smtp_port) as server:
        #     server.starttls()
        #     server.login(sender_email, sender_password)
        #     server.sendmail(sender_email, receiver_email, message.as_string())
        logger.info("알림 이메일 발송 완료")
        print("📧 이메일 알림이 발송되었습니다 (시뮬레이션)")
    except Exception as e:
        logger.error(f"이메일 발송 실패: {e}")

# 테스트 (실제 이메일은 발송하지 않음)
print("\n이메일 알림 기능 (시뮬레이션):")
send_alert_email(summary)

### 2. 대시보드 생성 (Matplotlib)

In [ ]:
# 주의: Jupyter에서 실행 시 matplotlib 필요
# pip install matplotlib

try:
    import matplotlib.pyplot as plt
    from collections import Counter
    
    def create_dashboard(summary, output_file="dashboard.png"):
        """
        분석 결과를 시각화하여 대시보드 생성
        
        Args:
            summary: 분석 결과 요약
            output_file: 출력 이미지 파일명
        """
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        fig.suptitle('로그 분석 대시보드', fontsize=16, fontweight='bold')
        
        # 1. 로그 레벨별 파이 차트
        level_counts = summary['level_counts']
        colors = {'ERROR': '#d32f2f', 'WARNING': '#f57c00', 'INFO': '#1976d2'}
        
        axes[0, 0].pie(
            level_counts.values(),
            labels=level_counts.keys(),
            autopct='%1.1f%%',
            colors=[colors.get(k, '#777') for k in level_counts.keys()],
            startangle=90
        )
        axes[0, 0].set_title('로그 레벨 분포')
        
        # 2. 날짜별 에러 트렌드
        daily_errors = {date: stats.get('ERROR', 0) 
                       for date, stats in summary['daily_stats'].items()}
        
        axes[0, 1].plot(list(daily_errors.keys()), list(daily_errors.values()), 
                       marker='o', color='#d32f2f', linewidth=2)
        axes[0, 1].set_title('날짜별 에러 추이')
        axes[0, 1].set_xlabel('날짜')
        axes[0, 1].set_ylabel('에러 수')
        axes[0, 1].tick_params(axis='x', rotation=45)
        axes[0, 1].grid(True, alpha=0.3)
        
        # 3. 레벨별 막대 그래프
        axes[1, 0].bar(
            level_counts.keys(),
            level_counts.values(),
            color=[colors.get(k, '#777') for k in level_counts.keys()]
        )
        axes[1, 0].set_title('레벨별 로그 수')
        axes[1, 0].set_ylabel('개수')
        axes[1, 0].grid(axis='y', alpha=0.3)
        
        # 4. 통계 요약 테이블
        axes[1, 1].axis('off')
        stats_text = f"""
        📊 전체 통계
        
        총 로그: {summary['total_logs']:,}개
        처리된 파일: {summary['files_processed']}개
        
        🔴 ERROR: {summary['error_count']}개
        🟡 WARNING: {summary['warning_count']}개
        🔵 INFO: {summary['info_count']}개
        """
        axes[1, 1].text(0.1, 0.5, stats_text, fontsize=12, 
                       verticalalignment='center', family='monospace')
        
        plt.tight_layout()
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        print(f"📊 대시보드 저장: {output_file}")
        plt.show()
    
    # 실행
    create_dashboard(summary)
    
except ImportError:
    print("⚠️  matplotlib이 설치되지 않았습니다.")
    print("설치: pip install matplotlib")

### 3. 스케줄링 (Schedule)

In [ ]:
# 주의: 실제 사용 시 schedule 패키지 필요
# pip install schedule

def scheduled_analysis_demo():
    """
    정기적으로 로그 분석 실행 (데모)
    """
    code_example = '''
import schedule
import time

def run_analysis():
    """분석 작업 실행"""
    bot = LogAnalyzerBot("server_logs")
    bot.run()

# 매 시간마다 실행
schedule.every().hour.do(run_analysis)

# 매일 오전 9시에 실행
schedule.every().day.at("09:00").do(run_analysis)

# 월요일마다 실행
schedule.every().monday.do(run_analysis)

print("스케줄러 시작...")
while True:
    schedule.run_pending()
    time.sleep(60)  # 1분마다 체크
    '''
    
    print("\n⏰ 스케줄링 예제 코드:")
    print(code_example)

scheduled_analysis_demo()

### 4. 실시간 모니터링 (Watchdog)

In [ ]:
# 주의: 실제 사용 시 watchdog 패키지 필요
# pip install watchdog

def realtime_monitoring_demo():
    """
    실시간 로그 파일 모니터링 (데모)
    """
    code_example = '''
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import time

class LogFileHandler(FileSystemEventHandler):
    """로그 파일 변경 감지 및 자동 분석"""
    
    def __init__(self, analyzer_bot):
        self.bot = analyzer_bot
    
    def on_modified(self, event):
        if event.src_path.endswith('.log'):
            print(f"로그 파일 변경 감지: {event.src_path}")
            # 즉시 분석 실행
            self.bot.run()
    
    def on_created(self, event):
        if event.src_path.endswith('.log'):
            print(f"새 로그 파일 생성: {event.src_path}")
            self.bot.run()

# 사용 예
bot = LogAnalyzerBot("server_logs")
event_handler = LogFileHandler(bot)
observer = Observer()
observer.schedule(event_handler, path="server_logs", recursive=False)

print("실시간 모니터링 시작...")
observer.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()

observer.join()
    '''
    
    print("\n👀 실시간 모니터링 예제 코드:")
    print(code_example)

realtime_monitoring_demo()

---
## 전체 통합 실행

In [ ]:
# 전체 프로세스를 하나의 셀에서 실행

print("\n" + "="*70)
print("🚀 로그 분석 자동화 봇 - 전체 프로세스 실행")
print("="*70)

# 1. 봇 생성 및 실행
bot = LogAnalyzerBot("server_logs", output_dir="reports")
success = bot.run()

if success:
    print("\n" + "="*70)
    print("📁 생성된 파일 목록:")
    print("="*70)
    
    # reports 디렉토리 내용 확인
    import os
    if os.path.exists("reports"):
        for file in os.listdir("reports"):
            file_path = os.path.join("reports", file)
            size = os.path.getsize(file_path)
            print(f"  📄 {file} ({size:,} bytes)")
    
    print("\n" + "="*70)
    print("✅ 모든 작업이 완료되었습니다!")
    print("="*70)
    
    print("\n다음 단계:")
    print("  1. reports/ 폴더에서 생성된 리포트 확인")
    print("  2. log_analyzer.log 파일에서 상세 로그 확인")
    print("  3. 필요시 확장 기능 추가 (이메일, 스케줄링 등)")
else:
    print("\n❌ 작업 실패. log_analyzer.log를 확인하세요.")

---
## 학습 정리 및 체크리스트

### 완료 체크리스트

- ✅ Task 1: 샘플 로그 파일 3개 생성 완료
- ✅ Task 2: glob으로 로그 파일 발견 성공
- ✅ Task 3: 정규식 패턴으로 로그 파싱 성공
- ✅ Task 4: 제너레이터로 메모리 효율적 처리 구현
- ✅ Task 5: LogAnalyzer 클래스 구현 완료
- ✅ Task 6: JSON 리포트 생성 및 콘솔 출력 성공
- ✅ Task 7: 완전 자동화 봇 구현 및 실행 성공
- ✅ 예외 처리가 모든 주요 메서드에 적용됨
- ✅ 로깅이 적절하게 구현됨
- ✅ 코드가 PEP 8 스타일 가이드를 따름

### 핵심 학습 내용

#### Day 1 개념 활용
- **Collections**: Counter로 레벨별 카운트, defaultdict로 날짜별 통계
- **Comprehensions**: 리스트/딕셔너리 컴프리헨션으로 간결한 데이터 처리

#### Day 2 개념 활용
- **Generators**: 메모리 효율적인 대용량 파일 처리
- **File I/O**: 로그 읽기, 리포트 저장
- **JSON**: 구조화된 데이터 저장 및 교환
- **Modules**: glob, re 등 표준 라이브러리 활용

#### Day 3 개념 활용
- **OOP**: LogAnalyzer, LogAnalyzerBot 클래스로 로직 캡슐화
- **Exception Handling**: try-except로 견고한 에러 처리
- **Logging**: 프로세스 추적 및 디버깅
- **Regex**: 로그 패턴 파싱
- **Glob**: 파일 패턴 매칭
- **Datetime**: 타임스탬프 처리 및 리포트 생성

### 실전 적용 포인트

1. **확장성**: 클래스 기반 설계로 쉽게 기능 추가 가능
2. **견고성**: 예외 처리와 로깅으로 안정적인 운영
3. **효율성**: 제너레이터로 대용량 데이터 처리
4. **유지보수성**: 명확한 책임 분리와 문서화
5. **자동화**: 반복 작업을 코드로 자동화하여 생산성 향상

---

## 축하합니다! 🎉

3일간의 Python 부트캠프를 성공적으로 완료하셨습니다!

이제 여러분은:
- Python 핵심 데이터 구조를 능숙하게 다룰 수 있습니다
- 메모리 효율적인 코드를 작성할 수 있습니다
- 객체지향 프로그래밍으로 견고한 시스템을 설계할 수 있습니다
- 실전 프로젝트를 자동화할 수 있습니다

계속해서 Python을 학습하고 실전에 적용해보세요! 💪